In [9]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [10]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, SystemMessage
from typing import TypedDict

llm = ChatOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-3.5-turbo"
)

system_message = SystemMessage(content="Eres una calculadora diseñada para resolver las operaciones básicas")


class State(TypedDict):
  first_number: int
  second_number: int
  result: int

In [11]:
GET_FIRST_NUMBER_NODE = "get_first_number_node"
GET_SECOND_NUMBER_NODE = "get_second_number_node"
GET_RESULT_NODE = "get_result_node"

In [12]:
def get_first_number_node(state):
  human_message = HumanMessage(
      content="Genera un número aleatorio del 1 al 10 y devuélvelo solo como un número, sin texto adicional.")
  messages = [system_message, human_message]
  ia_response = llm.invoke(messages)
  response = int(ia_response.content)
  state["first_number"] = int(response)
  print(f"first number: {state["first_number"]}")
  return state

def get_second_number_node(state):
  human_message = HumanMessage(
      content="Genera un número aleatorio del 1 al 10 y devuélvelo solo como un número, sin texto adicional.")
  messages = [system_message, human_message]
  ia_response = llm.invoke(messages)
  response = int(ia_response.content)
  state["second_number"] = int(response)
  print(f"second number: {state["second_number"]}")
  return state

In [17]:
workflow = StateGraph(State)

workflow.add_node(GET_FIRST_NUMBER_NODE, get_first_number_node)
workflow.add_node(GET_SECOND_NUMBER_NODE, get_second_number_node)

workflow.set_entry_point(GET_FIRST_NUMBER_NODE)
workflow.set_finish_point(GET_SECOND_NUMBER_NODE)

workflow.add_edge(GET_FIRST_NUMBER_NODE, GET_SECOND_NUMBER_NODE)


In [18]:
agent = workflow.compile()

In [19]:
initial_state = {"first_number": 0, "second_number": 0, "result": 0}
agent.invoke(initial_state)

first number: 7
second number: 8


{'first_number': 7, 'second_number': 8, 'result': 0}